In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

from data_utils import load_images
import torch

In [2]:
train_dataset = load_images("train", percent=1)
test_dataset = load_images("test", percent=1)
validation_dataset = load_images("validation")

In [ ]:
from neural_network import get_dataloader_from_image_dataset


# We get the data in the form of a PyTorch dataset

train_loader = get_dataloader_from_image_dataset(train_dataset, batch_size=64)
test_loader = get_dataloader_from_image_dataset(
    test_dataset, batch_size=64, is_test=True
)
validation_loader = get_dataloader_from_image_dataset(
    validation_dataset, batch_size=64, is_test=True
)

In [4]:
print(train_dataset.images.shape)

(12500, 100, 100, 3)


In [5]:
from neural_network import NeuralNetwork

model = NeuralNetwork()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

NUM_EPOCHS = 10
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

loss_fn = torch.nn.CrossEntropyLoss()

/usr/local/lib/python3.10/dist-packages/torch/cuda/__init__.py:789: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [ ]:
epoch_losses = []
model.train(True)
for i in range(NUM_EPOCHS):
    print(f"Starting epoch {i+1}/{NUM_EPOCHS}")
    running_loss = 0.0
    num_batches = 0

    for batch_idx, (image_batch, labels_batch) in enumerate(train_loader):
        image_batch = image_batch.to(device)
        labels_batch = labels_batch.to(device)

        # Change shape from (batch_size, 100, 100, 3) to (batch_size, 3, 100, 100)
        image_batch = image_batch.permute(0, 3, 1, 2)

        predictions = model(image_batch)
        loss = loss_fn(predictions, labels_batch)

        running_loss += loss.item()
        num_batches += 1

        # Backpropagation and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print(f"Epoch {i+1}/{NUM_EPOCHS}, Batch {batch_idx}, Loss: {loss.item()}")

    epoch_loss = running_loss / num_batches
    epoch_losses.append(epoch_loss)

Starting epoch 1/10
Epoch 1/10, Batch 0, Loss: 1.5111674070358276


Epoch 1/10, Batch 100, Loss: 0.774992823600769
Starting epoch 2/10
Epoch 2/10, Batch 0, Loss: 0.6075841784477234
Epoch 2/10, Batch 100, Loss: 0.49142181873321533
Starting epoch 3/10
Epoch 3/10, Batch 0, Loss: 0.5471571087837219
Epoch 3/10, Batch 100, Loss: 0.42165735363960266
Starting epoch 4/10
Epoch 4/10, Batch 0, Loss: 0.35133594274520874
Epoch 4/10, Batch 100, Loss: 0.2646517753601074
Starting epoch 5/10
Epoch 5/10, Batch 0, Loss: 0.2439163327217102
Epoch 5/10, Batch 100, Loss: 0.25568005442619324
Starting epoch 6/10
Epoch 6/10, Batch 0, Loss: 0.11183685064315796
Epoch 6/10, Batch 100, Loss: 0.2745908498764038
Starting epoch 7/10
Epoch 7/10, Batch 0, Loss: 0.08314410597085953
Epoch 7/10, Batch 100, Loss: 0.12523984909057617
Starting epoch 8/10
Epoch 8/10, Batch 0, Loss: 0.152690127491951
Epoch 8/10, Batch 100, Loss: 0.10186012089252472
Starting epoch 9/10
Epoch 9/10, Batch 0, Loss: 0.04734457656741142
Epoch 9/10, Batch 100, Loss: 0.05680951476097107
Starting epoch 10/10
Epoch 10/10

In [ ]:
correct = 0.0
test_loss = 0.0
size = len(validation_loader.dataset)  # numarul total de exemple din setul de validare
model.to(device)
model.eval()
predictions = []
with torch.no_grad():
    for image_batch, labels_batch in validation_loader:
        image_batch = image_batch.to(device)
        labels_batch = labels_batch.to(device)
        image_batch = image_batch.permute(0, 3, 1, 2)

        pred = model(
            image_batch
        )  # procesam imaginile folosind reteaua antrenata anterior

        predictions.append(pred)

        test_loss += loss_fn(pred, labels_batch).item()

        correct += (
            (pred.argmax(1) == labels_batch).type(torch.float).sum().item()
        )  # numaram cate exemple sunt corect clasificate

correct /= size
test_loss /= size
print(f"Accuracy: {(100*correct):>0.1f}%, Loss: {test_loss:>8f} \n")

predictions = torch.cat(predictions, dim=0)
predictions = predictions.argmax(dim=1).cpu().numpy()
print(predictions.shape)

Accuracy: 81.0%, Loss: 0.009528 

(1250,)


In [ ]:
correct = 0.0
test_loss = 0.0
size = len(validation_loader.dataset)  # numarul total de exemple din setul de validare
model.to(device)
model.eval()
predictions = []
with torch.no_grad():
    for image_batch, labels_batch in test_loader:
        image_batch = image_batch.to(device)
        image_batch = image_batch.permute(0, 3, 1, 2)
        pred = model(
            image_batch
        )  # procesam imaginile folosind reteaua antrenata anterior
        predictions.append(pred)
predictions = torch.cat(predictions, dim=0)
predictions = predictions.argmax(dim=1).cpu().numpy()
print(predictions)

In [ ]:
from data_utils import save_predictions_to_csv

ids = test_dataset.ids

save_predictions_to_csv(predictions, ids, "predictions2.csv")